In [5]:
from influxdb_client_3 import InfluxDBClient3
import pyarrow as pa
import duckdb

# Connect to InfluxDB 3
client = InfluxDBClient3(
    host="http://localhost:9000",
    token="apiv3_wfr_admin_token_change_in_production",
    database="WFR25"
)

In [6]:
# Check InfluxDB connection
try:
    # Simple query to test connection
    test_query = client.query("SELECT 1 as test")
    print("Connection to InfluxDB successful!")
    print("Test query result:", test_query)
except Exception as e:
    print(f"Connection failed: {e}")

Connection to InfluxDB successful!
Test query result: pyarrow.Table
test: int64 not null
----
test: [[1]]


In [23]:
# Query to find the highest historical value of signal 'INV_DC_Bus_Voltage'
max_value_table = client.query("""
    SELECT MAX("sensorReading") as max_value
    FROM "iox"."WFR25"
    WHERE "signalName" = 'INV_DC_Bus_Voltage'
""")

print("Max value query result:")
print(max_value_table)

Max value query result:
pyarrow.Table
max_value: double
----
max_value: [[407.6]]


In [8]:
# Query to get the latest 10 entries from WFR25
latest_entries = client.query("""
    SELECT time, "sensorReading", "signalName"
    FROM "iox"."WFR25"
    ORDER BY time DESC
    LIMIT 10
""")

print("Latest 10 entries:")
print(latest_entries)

Latest 10 entries:
pyarrow.Table
time: timestamp[ns] not null
sensorReading: double
signalName: string
----
time: [[2025-09-30 01:47:51.326999808,2025-09-30 01:47:51.326999808,2025-09-30 01:47:51.326999808,2025-09-30 01:47:51.326999808,2025-09-30 01:47:51.326999808,2025-09-30 01:47:51.326999808,2025-09-30 01:47:51.326999808,2025-09-30 01:47:51.316000000,2025-09-30 01:47:51.316000000,2025-09-30 01:47:51.316000000]]
sensorReading: [[0,0,0,0,0,0,0,0,0,-0.4]]
signalName: [["VCU_INV_Speed_Command","VCU_INV_Torque_Limit_Command","VCU_INV_Direction_Command","VCU_INV_Inverter_Discharge","VCU_INV_Torque_Command","VCU_INV_Speed_Mode_Enable","VCU_INV_Inverter_Enable","INV_Fast_Motor_Speed","INV_Fast_Torque_Command","INV_Fast_Torque_Feedback"]]


In [9]:
# Query to list available tables in the database
tables = client.query("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'iox'
""")

print("Available tables in iox schema:")
print(tables)

Available tables in iox schema:
pyarrow.Table
table_name: string not null
----
table_name: [["WFR25"]]


In [10]:
# Query to list all tables in the database
all_tables = client.query("""
    SELECT table_schema, table_name
    FROM information_schema.tables
""")

print("All tables:")
print(all_tables)

All tables:
pyarrow.Table
table_schema: string not null
table_name: string not null
----
table_schema: [["iox","system","system","system","system",...,"information_schema","information_schema","information_schema","information_schema","information_schema"]]
table_name: [["WFR25","distinct_caches","influxdb_schema","last_caches","parquet_files",...,"columns","df_settings","schemata","routines","parameters"]]


In [11]:
# Check the schema of the WFR25 table
schema_check = client.query("""
    SELECT *
    FROM "iox"."WFR25"
    LIMIT 1
""")

print("Table schema:")
print(schema_check.schema)
print("Sample data:")
print(schema_check)

Table schema:
canId: string
  -- field metadata --
  iox::column::type: 'iox::column_type::tag'
messageName: string
  -- field metadata --
  iox::column::type: 'iox::column_type::tag'
sensorReading: double
  -- field metadata --
  iox::column::type: 'iox::column_type::field::float'
signalName: string
  -- field metadata --
  iox::column::type: 'iox::column_type::tag'
time: timestamp[ns] not null
  -- field metadata --
  iox::column::type: 'iox::column_type::timestamp'
Sample data:
pyarrow.Table
canId: string
messageName: string
sensorReading: double
signalName: string
time: timestamp[ns] not null
----
canId: [["2014"]]
messageName: [["VCU_Front_Sensors_1"]]
sensorReading: [[0.48]]
signalName: [["A1"]]
time: [[2025-09-28 22:19:10.292999936]]


In [17]:
# Query to find rolling 5-second max for M4_Cell10_Voltage
rolling_max_5s = client.query("""
    SELECT 
        (EXTRACT(epoch from time) / 5)::bigint * 5 as bucket_epoch,
        MAX("sensorReading") as max_voltage
    FROM "iox"."WFR25"
    WHERE "signalName" = 'M4_Cell10_Voltage'
    GROUP BY bucket_epoch
    ORDER BY bucket_epoch
""")

print("Rolling 5-second max for M4_Cell10_Voltage:")
print(rolling_max_5s)

Rolling 5-second max for M4_Cell10_Voltage:
pyarrow.Table
bucket_epoch: int64
max_voltage: double
----
bucket_epoch: [[1759097945,1759097950,1759097955,1759097960,1759097965,...,1759196835,1759196840,1759196845,1759196850,1759196855]]
max_voltage: [[4.0325,4.0325,4.0324,4.0325,4.0324,...,3.3093000000000004,3.3091000000000004,3.3092,3.3092,3.309]]


In [25]:
voltage_data = client.query("""
    SELECT time, "sensorReading"
    FROM "iox"."WFR25"
    WHERE "signalName" = 'INV_DC_Bus_Voltage'
    ORDER BY time
""")

# Convert to pandas for analysis
import pandas as pd
df = voltage_data.to_pandas()
df.set_index('time', inplace=True)

# Calculate rolling median with 5-second window
rolling_median = df['sensorReading'].rolling('5s').median()

# Find the maximum median and its time
max_median = rolling_median.max()
max_time = rolling_median.idxmax()

# The period is from max_time - 5s to max_time
start_time = max_time - pd.Timedelta(seconds=5)
end_time = max_time

print(f"Highest median: {max_median}")
print(f"Period start: {start_time}")
print(f"Period end: {end_time}")

Highest median: 407.3
Period start: 2025-09-28 22:19:20.105999872
Period end: 2025-09-28 22:19:25.105999872


In [27]:
# Query voltage
voltage_data = client.query("""
    SELECT time, "sensorReading"
    FROM "iox"."WFR25"
    WHERE "signalName" = 'INV_DC_Bus_Voltage'
    ORDER BY time
""")

# Query current
current_data = client.query("""
    SELECT time, "sensorReading"
    FROM "iox"."WFR25"
    WHERE "signalName" = 'INV_DC_Bus_Current'
    ORDER BY time
""")

# Convert both to pandas
import pandas as pd

df_v = voltage_data.to_pandas().rename(columns={"sensorReading": "voltage"})
df_c = current_data.to_pandas().rename(columns={"sensorReading": "current"})

# Set timestamp index
df_v.set_index("time", inplace=True)
df_c.set_index("time", inplace=True)

# Merge on time (inner join to align timestamps)
df = pd.merge_asof(df_v.sort_index(), df_c.sort_index(), 
                   left_index=True, right_index=True, 
                   direction="nearest", tolerance=pd.Timedelta("50ms"))

# Compute power
df["power"] = df["voltage"] * df["current"]

# Rolling average over 5 seconds
rolling_avg = df["power"].rolling("5s").mean()

# Find the maximum sustained 5s power
max_avg = rolling_avg.max()
max_time = rolling_avg.idxmax()

# The sustained period is the 5s ending at max_time
start_time = max_time - pd.Timedelta(seconds=5)
end_time = max_time

print(f"Highest sustained 5s avg power: {max_avg:.2f} W")
print(f"Period start: {start_time}")
print(f"Period end: {end_time}")

Highest sustained 5s avg power: 29050.29 W
Period start: 2025-09-28 22:50:15.176000
Period end: 2025-09-28 22:50:20.176000
